Trước khi run pip, cài đặt các thư viện bằng câu lệnh sau
```
pip install dash
```

In [1]:
# Cài đặt các thư viện cần thiết
import dash
from dash import dcc
from dash import html
import pandas as pd
import plotly.graph_objs as go
from dash.dependencies import Input, Output
from keras.models import load_model as keras_load_model
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))

import numpy as np

# Khởi tạo ứng dụng Dash
app = dash.Dash()
server = app.server

In [2]:
# Lấy dữ liệu EURUSD từ MetaTrader5
import MetaTrader5 as mt
import pandas as pd
from datetime import datetime

mt.initialize()

login = 84193298
password = "-8WyMrVz"
server = "MetaQuotes-Demo"

mt.login(login, password, server)

def get_data_FX(symbol, timeframe, rows):
    ohlc_data = pd.DataFrame(mt.copy_rates_from_pos(symbol, timeframe, 0, rows))
    ohlc_data['time'] = ohlc_data.apply(lambda x: datetime.fromtimestamp(x["time"]), axis=1)
    ohlc_data = ohlc_data.rename(columns={'time': 'Date', 'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'real_volume': 'Volume'})
    ohlc_data.set_index("Date")
    return ohlc_data

dataframe = get_data_FX('EURUSD', mt.TIMEFRAME_M1,2000)

dataframe.head()

,Date,Open,High,Low,Close,tick_volume,spread,Volume
0,2024-07-18 21:38:00,1.09299,1.09307,1.09299,1.09303,28,1,0
1,2024-07-18 21:39:00,1.09303,1.09309,1.09303,1.09308,19,1,0
2,2024-07-18 21:40:00,1.09308,1.09308,1.09300,1.09300,21,1,0
3,2024-07-18 21:41:00,1.09298,1.09310,1.09298,1.09300,34,1,0
4,2024-07-18 21:42:00,1.09300,1.09305,1.09299,1.09300,23,1,0


In [3]:
# Sắp xếp theo ngày, lấy cột ngày làm index
dataframe["Date"]=pd.to_datetime(dataframe.Date,format="%Y-%m-%d")
dataframe.index=dataframe['Date'].values

dataframe=dataframe.sort_index(ascending=True,axis=0)

# Tạo dataframe mới chứa dữ liệu cần thiết
temp_df=pd.DataFrame(index=range(0,len(dataframe)),columns=['Date','Close', 'ROC'])

for i in range(0,len(dataframe)):
    temp_df.loc[i, "Date"]=dataframe['Date'].iloc[i]
    temp_df.loc[i, "Close"]=dataframe["Close"].iloc[i]

# Tính chỉ số ROC
temp_df.loc[0, "ROC"] = None
for i in range(1, len(dataframe)):
    temp_df.loc[i, "ROC"] = (temp_df.loc[i, "Close"] - temp_df.loc[i-1, "Close"]) / temp_df.loc[i-1, "Close"]

temp_df.index=temp_df['Date'].values
temp_df.drop("Date",axis=1,inplace=True)

# Thêm 1 dòng vào cuối dữ liệu
last_date = temp_df.index[-1]
last_date = last_date + pd.DateOffset(minutes=1)
temp_df.loc[last_date] = [None for _ in range(len(temp_df.columns))]

dataframe = temp_df

In [4]:
# Lấy mẫu từ dữ liệu để đưa ra dự đoán giá cổ phiếu bằng mô hình LSTM, RNN, XGB

model_names = ['LSTM', 'RNN', 'XGB']
col_names = ['Close', 'ROC']

#model_names = ['XGB']
#col_names = ['Close']

train=dataframe[:-100]
valid=dataframe[-100:]

for model_name in model_names:
    if model_name == 'LSTM' or model_name == 'RNN':
        model = keras_load_model(f"model/{model_name.lower()}_close_model.keras")
    elif model_name == 'XGB':
        model = xgb.Booster()
        model.load_model(f"model/xgboost_close_model.json")

    for col_name in col_names:
        inputs=temp_df[len(temp_df)-len(valid)-60:][col_name].values
        inputs=inputs.reshape(-1,1)

        scaler=MinMaxScaler(feature_range=(0,1))
        inputs=scaler.fit_transform(inputs)

        X_test=[]
        for i in range(60,inputs.shape[0]):
            X_test.append(inputs[i-60:i,0])
        X_test=np.array(X_test)


        if model_name == 'LSTM' or model_name == 'RNN':
            X_test_3D=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
            output=model.predict(X_test_3D)
            output=scaler.inverse_transform(output)
        elif model_name == 'XGB':
            X_test_DMatrix=xgb.DMatrix(X_test)
            output=model.predict(X_test_DMatrix).reshape(-1, 1)
            output=scaler.inverse_transform(output)

        # Add column
        dataframe.insert(dataframe.shape[1], f'Predictions_{col_name}_{model_name}', np.nan)
        for i in range(0, len(output)):
            dataframe.loc[dataframe.index[-len(output)+i], f'Predictions_{col_name}_{model_name}'] = output[i][0]

dataframe.tail()


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 124ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


,Close,ROC,Predictions_Close_LSTM,Predictions_ROC_LSTM,Predictions_Close_RNN,Predictions_ROC_RNN,Predictions_Close_XGB,Predictions_ROC_XGB
2024-07-20 06:55:00,1.08816,-0.000028,1.088116,0.000004,1.088214,0.000109,1.088164,0.000082
2024-07-20 06:56:00,1.08817,0.000009,1.088125,0.000005,1.088145,-0.000072,1.088141,-0.000024
2024-07-20 06:57:00,1.08831,0.000129,1.088136,0.000007,1.088174,0.000017,1.088159,0.000011
2024-07-20 06:58:00,1.08839,0.000074,1.088171,0.000029,1.088286,0.000078,1.088247,0.000087
2024-07-20 06:59:00,None,None,1.088226,0.000047,1.088370,0.000057,1.088261,0.000071


In [6]:
app.layout = html.Div([

    html.H1("Forex Price Analysis Dashboard", style={"textAlign": "center"}),

    dcc.Tabs(id="tabs", children=[

        dcc.Tab(label='EURUSD Actual closing price',children=[
			html.Div([
				html.H2("Actual closing price",style={"textAlign": "center"}),
				dcc.Graph(
					id="Actual Data",
					figure={
						"data":[
							go.Scatter(
								x=dataframe.index,
								y=dataframe["Close"],
								mode='markers'
							)

						],
						"layout":go.Layout(
							title='scatter plot',
							xaxis={'title':'Date'},
							yaxis={'title':'Closing Rate'}
						)
					}

				),
			])
        ]),

        dcc.Tab(label='EURUSD Prediction closing price', children=[
            html.Div([
                html.H1("Prediction closing price",
                        style={'textAlign': 'center'}),

                dcc.Dropdown(id='my-dropdown',
                             options=[{'label': 'LSTM', 'value': 'LSTM'},
                                      {'label': 'RNN','value': 'RNN'},
                                      {'label': 'XGBoost', 'value': 'XGB'}],
                             multi=True,value=['LSTM'],
                             style={"display": "block", "margin-left": "auto",
                                    "margin-right": "auto", "width": "60%"}),
                dcc.Graph(id='prediction'),

            ]),
        ]),

        dcc.Tab(label='EURUSD Prediction ROC', children=[
            html.Div([
                html.H1("Prediction ROC",
                        style={'textAlign': 'center'}),

                dcc.Dropdown(id='my-dropdown',
                             options=[{'label': 'LSTM', 'value': 'LSTM'},
                                      {'label': 'RNN','value': 'RNN'},
                                      {'label': 'XGBoost', 'value': 'XGB'}],
                             multi=True,value=['LSTM'],
                             style={"display": "block", "margin-left": "auto",
                                    "margin-right": "auto", "width": "60%"}),
                dcc.Graph(id='prediction_ROC'),

            ]),
        ])
        
    ])
])

@app.callback(Output('prediction', 'figure'),
              [Input('my-dropdown', 'value')])
def update_graph(selected_dropdown):
    dropdown = {"LSTM": "LSTM","RNN": "RNN", "XGB": "XGBoost"}
    trace1 = []
    trace2 = []
    
    trace1.append(
        go.Scatter(
            x=dataframe.index, 
            y=dataframe["Close"], 
            mode='lines', 
            opacity=0.6, 
            name="Actual Close Price", 
            marker_color='red',
            textposition='bottom center'
        )
    )

    for model in selected_dropdown:
        trace2.append(
            go.Scatter(
                x=dataframe.index,
				y=dataframe["Predictions_Close_" + model],
                mode='lines', 
                opacity=0.7,
                name=f'Predictions_{model}',
                textposition='bottom center'
            )
        )

    traces = [trace1, trace2]
    data = [val for sublist in traces for val in sublist]

    figure = {
        'data': data,
        'layout': go.Layout(colorway=
                            ["#FF0000", '#00FF00', '#00CC33', '#009966', '#006699', '#0033CC'],
        height=600,
        title=f"Close price for {', '.join(str(dropdown[i]) for i in selected_dropdown)} Over Time",
        xaxis={"title":"Date",
            'rangeselector': {
                'buttons': list([
                    {'count': 1, 'label': '1H','step': 'hour','stepmode': 'backward'},
                    {'count': 1, 'label': '1D','step': 'day','stepmode': 'backward'},
                    {'count': 7, 'label': '1W','step': 'day','stepmode': 'backward'},
                    {'count': 1, 'label': '1M','step': 'month','stepmode': 'backward'},
                    {'count': 6, 'label': '6M','step': 'month', 'stepmode': 'backward'},
                    {'step': 'all'}
                ])},
                'rangeslider': {'visible': True}, 'type': 'date'
            },
            yaxis={"title":"Price (USD)"})
    }
    return figure


@app.callback(Output('prediction_ROC', 'figure'),
              [Input('my-dropdown', 'value')])
def update_graph(selected_dropdown):
    dropdown = {"LSTM": "LSTM","RNN": "RNN", "XGB": "XGBoost"}
    trace1 = []
    trace2 = []
    
    trace1.append(
        go.Scatter(
            x=dataframe.index, 
            y=dataframe["ROC"], 
            mode='lines', 
            opacity=0.6, 
            name="Actual ROC", 
            marker_color='red',
            textposition='bottom center'
        )
    )

    for model in selected_dropdown:
        trace2.append(
            go.Scatter(
                x=dataframe.index,
				y=dataframe["Predictions_ROC_" + model],
                mode='lines', 
                opacity=0.7,
                name=f'Predictions_{model}',
                textposition='bottom center'
            )
        )

    traces = [trace1, trace2]
    data = [val for sublist in traces for val in sublist]

    figure = {
        'data': data,
        'layout': go.Layout(colorway=
                            ["#FF0000", '#00FF00', '#00CC33', '#009966', '#006699', '#0033CC'],
        height=600,
        title=f"ROC for {', '.join(str(dropdown[i]) for i in selected_dropdown)} Over Time",
        xaxis={"title":"Date",
            'rangeselector': {
                'buttons': list([
                    {'count': 1, 'label': '1H','step': 'hour','stepmode': 'backward'},
                    {'count': 1, 'label': '1D','step': 'day','stepmode': 'backward'},
                    {'count': 7, 'label': '1W','step': 'day','stepmode': 'backward'},
                    {'count': 1, 'label': '1M','step': 'month','stepmode': 'backward'},
                    {'count': 6, 'label': '6M','step': 'month', 'stepmode': 'backward'},
                    {'step': 'all'}
                ])},
                'rangeslider': {'visible': True}, 'type': 'date'
            },
            yaxis={"title":"ROC"})
    }
    return figure


if __name__=='__main__':
	app.run_server(debug=True)
    

---------------------------------------------------------------------------
DuplicateIdError                          Traceback (most recent call last)
DuplicateIdError: Duplicate component id found in the initial layout: `my-dropdown`

